# Add Folders to Path

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add integration folder to current working path
sys.path.append(parent_directory+"/Src/integration")

# add utils folder to current working path
sys.path.append(parent_directory+"/Src/utils")

# add EllipticLCS folder to current working path
sys.path.append(parent_directory+"/Demos/EllipticLCS")

CPU times: user 125 µs, sys: 97 µs, total: 222 µs
Wall time: 147 µs


In [2]:
# Suppress numba warnings arising from computation falling back into object mode.
from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)

# Overview

In the following notebok we extract elliptic LCS in the Agulhas region from the AVISO dataset. The notebook is structured as follows:

* Import data from the file 'Agulhas_AVISO.mat' stored in the folder 'Data'
<br />
* Define computational parameters (such as the number of cores) and variables
 <br />
* Define spatio-temporal domain over which to compute the elliptic LCS.
 <br />
* Interpolate velocity from the (discrete) gridded data using a cubic spline interpolation <br />
* Compute rate of strain tensor $ \mathbf{S}(\mathbf{x}, t) $ over meshgrid of initial conditions '$ \mathbf{x} $' at time '$t$' <br />
* Compute elliptic LCS as closed null geodesics of $ \mathbf{C}_{t_0}^t(\mathbf{x}, t)-\mu I $.

# Import Data

We start by import the AVISO-data from the file 'Agulhas_AVISO.mat' stored in the folder 'Data'.

In [3]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../Data/Aviso/processed_data/Agulhas_AVISO.mat')

U = mat_file['u'][:,:,:]
V = mat_file['v'][:,:,:]
x = mat_file['x']
y = mat_file['y']
time_data = mat_file['t'][:,:]

CPU times: user 18 ms, sys: 12.6 ms, total: 30.6 ms
Wall time: 29.5 ms


# Data/Parameters for Dynamical System

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 25

# time resolution of data
dt_data = time_data[0, 1]-time_data[0,0]

# periodic boundary conditions
periodic_x = False
periodic_y = False
periodic = [periodic_x, periodic_y]

# unsteady velocity field
bool_unsteady = True

# defined domain
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## compute meshgrid of dataset
X, Y = np.meshgrid(x, y)

## resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

# Spatio-Temporal Domain of Dynamical System

In [5]:
%%time
# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 25

# time step-size
dt = .1

time = np.arange(t0, tN+dt, dt)

# domain boundary (in degrees)
xmin = -3
xmax = 1
ymin = -32
ymax = -24

# spacing of meshgrid (in degrees)
dx = 0.02
dy = 0.02

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

CPU times: user 650 µs, sys: 818 µs, total: 1.47 ms
Wall time: 751 µs


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant. 

In [6]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, time_data, method = "cubic")

CPU times: user 123 ms, sys: 19.5 ms, total: 143 ms
Wall time: 142 ms


# Cauchy Green (CG) strain tensor over meshgrid of initial conditions

The rate of strain tensor $ S(\mathbf{x}, t) $ at time $ t $ is computed by iterating over meshgrid. The method *DS._spin_tensor(x, t)* computes the rate of strain tensor at point $ \mathbf{x} $  at time $ t $ by using an auxiliary meshgrid. 'aux_grid' specifies the ratio between the auxiliary grid and the original meshgrid. This parameter is generally chosen to be between $ [\dfrac{1}{10}, \dfrac{1}{100}] $. The computations are parallelized.

In [ ]:
%%time
# Import package for progress bar
from tqdm.notebook import tqdm

# Import package for parallel computing
from joblib import Parallel, delayed

# Import gradient of velocity function
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# Import Rate of Strain function
from ipynb.fs.defs.CauchyGreen import CauchyGreen

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .1 # [1/10, 1/100]
aux_grid = [np.around(aux_grid_ratio*(X_domain[0, 1]-X_domain[0, 0]), 5), np.around(aux_grid_ratio*(Y_domain[1, 0]-Y_domain[0, 0]), 5)]

def parallel_C(i):
    
    C_parallel = np.zeros((X_domain.shape[1], 2, 2))
    
    for j in range(C_parallel.shape[0]):
        
        x = np.array([X_domain[i,j], Y_domain[i,j]])
        
        # Compute gradient of flowmap
        gradFmap = gradient_flowmap(time, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
        
        # gradFmap has shape (2, 2, len(time)) --> we need gradient of flow map from t0 to tN
        gradFmap_t0_tN = gradFmap[:,:,-1]
        
        # Compute CG tensor
        C_parallel[j, :, :] = CauchyGreen(gradFmap_t0_tN)
    
    return C_parallel

C = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_C)(i) for i in tqdm(range(X_domain.shape[0]))))

  0%|          | 0/401 [00:00<?, ?it/s]

# Compute Tensorfield Properties

We now compute the properties of the rate of strain tensor 'S' such as the eigenvalues 's1', 's2' and eigenvectors 'eigenv1', 'eigenv2'. Furthermore, we also need the spatial derivatives of the elements of 'S'.

In [ ]:
# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS
        
# Compute gradients of elements of rate of strain tensor
C11 = np.nan_to_num(C[:,:,0,0], nan=0.0)
C12 = np.nan_to_num(C[:,:,0,1], nan=0.0)
C22 = np.nan_to_num(C[:,:,1,1], nan=0.0)
    
# Interpolate elements of rate of strain tensor (per default: cubic)
interp_C11 = RBS(Y_domain[:,0], X_domain[0,:], C11)
interp_C12 = RBS(Y_domain[:,0], X_domain[0,:], C12)
interp_C22 = RBS(Y_domain[:,0], X_domain[0,:], C22)

# Initialize arrays
C11x = np.zeros((C.shape[0], C.shape[1]))*np.nan
C11y = np.zeros((C.shape[0], C.shape[1]))*np.nan
C12x = np.zeros((C.shape[0], C.shape[1]))*np.nan
C12y = np.zeros((C.shape[0], C.shape[1]))*np.nan
C22x = np.zeros((C.shape[0], C.shape[1]))*np.nan
C22y = np.zeros((C.shape[0], C.shape[1]))*np.nan
    
# Define auxilary meshgrid
rho_x = aux_grid[0]
rho_y = aux_grid[1]
        
#iterate over meshgrid
for i in range(X_domain.shape[0]):
        
    for j in range(Y_domain.shape[1]):
            
        x = [X_domain[i,j], Y_domain[i, j]]
        
        # compute derivatives using auxiliary grid and finite-differencing
        C11x[i, j] = (interp_C11(x[1], x[0]+rho_x)[0][0]-interp_C11(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        C11y[i, j] = (interp_C11(x[1]+rho_y, x[0])[0][0]-interp_C11(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
        
        C12x[i, j] = (interp_C12(x[1], x[0]+rho_x)[0][0]-interp_C12(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        C12y[i, j] = (interp_C12(x[1]+rho_y, x[0])[0][0]-interp_C12(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            
        C22x[i, j] = (interp_C22(x[1], x[0]+rho_x)[0][0]-interp_C22(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        C22y[i, j] = (interp_C22(x[1]+rho_y, x[0])[0][0]-interp_C22(x[1]-rho_y, x[0])[0][0])/(2*rho_y)

# Interpolate $ \dot{\phi}(x, y, \phi)$ 

In [ ]:
%%time
from ipynb.fs.defs.phi_prime import _phi_prime

# Interpolant phi_phrime, interpolant indicating domain of existence (= denominator of phi_prime)
interp_phi_prime, interp_DOE = _phi_prime(X_domain, Y_domain, C11, C12, C22, C11x, C11y, C12x, C12y, C22x, C22y)

# Find closed null-geodesics of $ C − \mu I $

Elliptic LCSs are closed null geodesics of the one-parameter family of Lorentzian metrics $ C_{t_0}^{t_N}(\mathbf{x}_0) − \lambda $. $ \lambda ∈ \mathbb{R} $ denotes the tangential stretch rate along an elliptic LCS. For perfectly coherent elliptic LCS with no tangential stretching (=filamentation) it holds that $ \mu = 0 $.  

In [ ]:
%%time
# Import function to extract closed null geodesics
from ipynb.fs.defs.closed_null_geodesics import _closed_null_geodesics

# extract elliptic LCS from closed null geodesics
from ipynb.fs.defs.elliptic_LCS import _elliptic_LCS

# define lambda range
lam_min = 0.9
lam_max = 1.1
n_lam = 5
lam_range = np.linspace(lam_min, lam_max, n_lam)

# parameter specifying sparsity of initial conditions. 
# Increasing this parameter reduces the number of initial conditions and thereby reduces the computational time involved.
# For maximal accuracy set sparse_ic = 1. This implies that we use the resolution of the meshgrid. 
# This value should always be at least 1.
sparse_ic = 1

# threshold distance for periodicity of trajectory
d_threshold = dx

# compute Elliptic LCS
elliptic_LCS = []

# Initial conditions of mu
init_lam_range = []

for lam in lam_range:
    
    print("Compute elliptic LCS for lambda =", np.around(lam, 3))
    
    # Define list storing x, y coordinates of elliptic LCS
    x_elliptic, y_elliptic = [], []
    
    # Find all closed null geodesics
    closed_null_geodesics, init_lam = _closed_null_geodesics(X_domain, Y_domain, lam, interp_phi_prime, d_threshold, C11, interp_DOE, sparse_ic, Ncores)
    
    # Append initial conditions to list 
    init_lam_range.append(init_lam)
        
    # Extract elliptic LCS
    x_elliptic, y_elliptic = _elliptic_LCS(closed_null_geodesics)
           
    elliptic_LCS.append([x_elliptic, y_elliptic])

In [ ]:
######################################## Plot elliptic LCS ########################################

# import plotting library
import matplotlib.pyplot as plt
import matplotlib as mpl

# Define figure/axes
fig = plt.figure(figsize = (16, 10), dpi = 300)
ax = plt.axes()

# Define norm/cmap
norm = mpl.colors.Normalize(vmin=lam_range.min(), vmax=lam_range.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap='gist_rainbow_r')

# Iterate over all elliptic LCS and plot
for i in range(len(elliptic_LCS)):
    
    ax.scatter(init_lam_range[i][0], init_lam_range[i][1], s = 1, color=cmap.to_rgba(lam_range[i]))
    
    if elliptic_LCS[i][0] is not None:
    
        for j in range(len(elliptic_LCS[i][0])):
        
            if elliptic_LCS[i][0][j] is not None:
        
                ax.plot(elliptic_LCS[i][0][j], elliptic_LCS[i][1][j], c=cmap.to_rgba(lam_range[i]), linewidth = 2)

# Set axis limits
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

# Set axis labels
ax.set_xlabel("long (deg)", fontsize = 16)
ax.set_ylabel("lat (deg)", fontsize = 16)

# Colorbar
cbar = fig.colorbar(cmap, ticks=[mu_range.min(), 0, mu_range.max()])
cbar.ax.set_ylabel(r'$ \lambda $', rotation = 0, fontsize = 16)

# Title
ax.set_title("Elliptic LCS", fontsize = 20)
plt.show();